In [52]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill

## 拿資料

In [36]:
df_b = pd.read_excel('籌碼分點_20240109.xlsx')
df_b

,代號,名稱,年月日,券商代碼,券商名稱,買進股數(股),賣出股數(股),淨買賣股數(股),買進金額(千元),賣出金額(千元),淨買賣金額(千元),買進均價,賣出均價,營業額,市佔率
0,1101,台泥,2024/01/08,1021,合庫台中,15050,5000,10050,517.12,171.80,345.32,34.36,34.36,688.92,0.18
1,1101,台泥,2024/01/08,1022,合庫台南,2251,100000,-97749,77.31,3440.00,-3362.69,34.34,34.40,3517.31,0.92
2,1101,台泥,2024/01/08,1023,合庫高雄,1000,1000,0,34.35,34.30,0.05,34.35,34.30,68.65,0.02
3,1101,台泥,2024/01/08,1024,合庫嘉義,2300,0,2300,78.92,0.00,78.92,34.31,NaN,78.92,0.02
4,1101,台泥,2024/01/08,1025,合庫基隆,3000,3000,0,103.05,103.20,-0.15,34.35,34.40,206.25,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397436,9962,有益,2024/01/08,9A9K,永豐三重,1000,2000,-1000,18.80,37.25,-18.45,18.80,18.63,56.05,0.15
397437,9962,有益,2024/01/08,9A9r,永豐北高,0,5000,-5000,0.00,92.25,-92.25,NaN,18.45,92.25,0.25
397438,9962,有益,2024/01/08,9A9s,永豐彰化,6000,0,6000,111.60,0.00,111.60,18.60,NaN,111.60,0.30
397439,9962,有益,2024/01/08,9A9U,永豐中正,50000,0,50000,925.00,0.00,925.00,18.50,NaN,925.00,2.50


In [37]:
df_vol = pd.read_excel('vol.xlsx')
df_vol = df_vol[['代號', '成交量(千股)']]
df_vol

,代號,成交量(千股)
0,1101,5581
1,1102,1995
2,1103,151
3,1104,358
4,1108,1448
...,...,...
1808,9951,281
1809,9955,151
1810,9958,2360
1811,9960,24


## 分點 list

In [ ]:
lst_bname = []

In [48]:
# 排除 (外資) (總公司)
lst_bname_foreign = ['摩根大通', '上海匯豐' ,'美林', '台灣摩根', '台灣匯立', '法銀巴黎', '花旗環球', '美林', '美商高盛', '港商野村', '港麥格理', '瑞銀']
lst_bname_corporate = ['元大', '兆豐', '元富', '台新', '永豐金', '玉山', '合庫', '亞東', '康和', '統一', '凱基', '凱基台北', '富邦', '富邦經紀', '群益','華南永昌'
                      ,'中國信託', '臺銀', '宏遠', '彰銀']
lst_exclude = lst_bname_foreign + lst_bname_corporate

# 計算

In [42]:
df = df_b.copy()
df = df[['代號', '名稱', '年月日', '券商代碼', '券商名稱', '淨買賣股數(股)', '淨買賣金額(千元)']]
df['淨買賣金額(千元)'] = round(df['淨買賣金額(千元)']/10,1)
df.rename(columns={'淨買賣金額(千元)': '淨買賣金額(萬)'}, inplace=True)
df = df[df['淨買賣金額(萬)']>100]
df = df.sort_values(['券商名稱','淨買賣金額(萬)'],ascending=False).reset_index(drop=True)

# merge 成交量
df = df.merge(df_vol,on=['代號'],how='left')
df['buy_proportion'] = round(df['淨買賣股數(股)']/1000/df['成交量(千股)']*100,2)
df

,代號,名稱,年月日,券商代碼,券商名稱,淨買賣股數(股),淨買賣金額(萬),成交量(千股),buy_proportion
0,3035,智原,2024/01/08,5321,高橋龍潭,14800,569.6,19058,0.08
1,2388,威盛,2024/01/08,5321,高橋龍潭,33000,521.8,50684,0.07
2,2317,鴻海,2024/01/08,5321,高橋龍潭,41000,420.9,35053,0.12
3,3017,奇鋐,2024/01/08,5321,高橋龍潭,10500,321.4,18053,0.06
4,2376,技嘉,2024/01/08,5321,高橋龍潭,13000,315.8,6315,0.21
...,...,...,...,...,...,...,...,...,...
16649,4904,遠傳,2024/01/08,8960,上海匯豐,21000,167.1,1362,1.54
16650,2886,兆豐金,2024/01/08,8960,上海匯豐,38000,149.7,18003,0.21
16651,2029,盛餘,2024/01/08,8960,上海匯豐,43000,124.3,349,12.32
16652,3105,穩懋,2024/01/08,8960,上海匯豐,8000,120.8,3811,0.21


In [51]:
df[(df['buy_proportion']>5)&(df['淨買賣金額(萬)']>1000)&(~df['券商名稱'].isin(lst_exclude))].to_excel('check.xlsx')


In [59]:
df_out = df[(df['淨買賣金額(萬)']>500)&(~df['券商名稱'].isin(lst_exclude))].copy()
df_out = df_out.sort_values(['淨買賣金額(萬)'],ascending=False).reset_index(drop=True)

# 创建一个新的Excel工作簿
wb = Workbook()
ws = wb.active
red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
# for i in range(len(df_out)):
#     value = df_out.at[i,'淨買賣金額(萬)']
#     col_buy = df_out.columns.get_loc('淨買賣金額(萬)')
#     # cell = ws.cell(row=i, column=c_idx, value=value)
#     cell = ws.cell(row=i+1, column=col_buy+1, value=value)
#     if value > 4000:
#         cell.fill = red_fill

for r_idx, value in enumerate(df['淨買賣金額(萬)'], start=1):
    cell = ws.cell(row=r_idx, column=1, value=value)  # 这里的列索引为1表示第一列，根据实际情况调整
    if value > 4000:
        cell.fill = red_fill
        
wb.save('output.xlsx')

In [31]:
df[df['券商名稱']=='凱基信義']

,代號,名稱,年月日,券商代碼,券商名稱,淨買賣股數(股),淨買賣金額(萬),成交量(千股),buy_proportion
11772,3661,世芯-KY,2024/01/08,9216,凱基信義,31000,10715.5,1772,1.75
11773,2498,宏達電,2024/01/08,9216,凱基信義,1497600,8002.8,43799,3.42
11774,3035,智原,2024/01/08,9216,凱基信義,86975,3379.7,19058,0.46
11775,3017,奇鋐,2024/01/08,9216,凱基信義,83005,2520.7,18053,0.46
11776,4147,中裕,2024/01/08,9216,凱基信義,283000,2486.8,1816,15.58
11777,3376,新日興,2024/01/08,9216,凱基信義,117000,1470.8,5152,2.27
11778,3207,耀勝,2024/01/08,9216,凱基信義,163000,1353.8,999,16.32
11779,2707,晶華,2024/01/08,9216,凱基信義,55000,1223.9,963,5.71
11780,3036,文曄,2024/01/08,9216,凱基信義,105000,1196.8,2938,3.57
11781,3443,創意,2024/01/08,9216,凱基信義,6000,1006.5,1318,0.46
